# Simple Merge

In [1]:
# full code

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.23
"""

# import
import time
import random
import os
import shutil

#
print("Process Start.")
start_time = time.time()

# iterations
NUM_SAMPLES = 1000

# def
alphabet_samples = "abcdefghizklmnopqrstuvwxyz1234567890"

def random_string(length):
    result = ""
    for i in range(length):
        result += random.choice(alphabet_samples)
    return result

# def
first_name_samples = "김이박최정강조윤장임"
middle_name_samples = "민서예지도하주윤채현지"
last_name_samples = "준윤우원호후서연아은진"

def random_name():
    result = ""
    result += random.choice(first_name_samples)
    result += random.choice(middle_name_samples)
    result += random.choice(last_name_samples)
    return result

# os
if os.path.exists("personal_info"):
    shutil.rmtree("personal_info")
os.mkdir("personal_info")

# creating files
for i in range(NUM_SAMPLES):
    name = random_name()

    filename = "personal_info/" + str(i) + "_" + name + ".txt"
    
    outfile = open(filename, 'w')

    outfile.write("name : " + name + "\n")
    outfile.write("age : " + str(time.time())[-2:] + "\n")
    outfile.write("e-mail : " + random_string(8) + "@bhban.com" + "\n") # outfile = write("e-mail :" + random_string(8) + "@bhban.com\n")
    outfile.write("division :" + random_string(3) + "\n")
    outfile.write("telephone : 010-" + str(time.time())[-4:] + str(time.time())[-6:-2] + "\n")
    outfile.write("sex : " + random.choice(["male", "female"]))

    outfile.close()

#
print("Process Done.")
end_time = time.time()
print("Peter's code: " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code: 0.1101078987121582 seconds.


In [2]:
# full code

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.24
"""

# import
import time
import os

#
directory = "personal_info"
outfile_name = "simple_merged_ID.csv"

#
out_file = open(outfile_name, 'w')
input_files = os.listdir(directory)

#
print("Process Start")
start_time = time.time()

for filename in input_files:
    # exception
    if ".txt" not in filename:
        continue

    #
    file = open(directory + "/" + filename)

    content = file.read()

    out_file.write(content + "\n\n")

    file.close()

out_file.close()

#
print("Process Done.")
end_time = time.time()
print("Peter's code: " + str(end_time - start_time) + " seconds.")

Process Start
Process Done.
Peter's code: 0.024367094039916992 seconds.


## Summary
- ```outfile_name``` 의 확장자만 ```".txt"```에서 ```".csv"```로 변경

## MS Excel Encoding
- 엑셀 > 데이터 > 데이터 가져오기 (텍스트/CSV) > 열기(가져오기) > 파일원본 65001:유니코드(UTF-8) > 로드
- [참고자료](https://www.lesstif.com/life/ms-excel-utf-8-csv-54952504.html)
- [참고자료](https://www.unisender.com/en/support/contact-lists/import-contact/charset-utf-8/)

# Merge into CSV

## RPA code design
- 목표 정하기
  1. 결과물을 저장할 빈 CSV 파일을 생성한다.
  1. CSV 파일에 헤더(Header)를 삽입한다.
  1. personal_info 폴더 내부의 내용물을 하나씩 읽어온다.
  1. 읽어온 내용물을 보기 좋게 다듬는다.
  1. 다듬은 내용물을 결과 파일에 옮겨 적는다.
  1. 내용물 작성이 완료되었다면 파일을 종료한다.
- 목표를 달성하는 데 필요한 작업 쪼개기
  1. personal_info 폴더 안의 내용물을 목록으로 만들기
  1. 목록 안의 내용물을 처음부터 끝까지 훑으며, 한 개씩 파일을 읽어오기
  1. 읽어온 파일에서 내용물 뽑아내기
  1. 파일에 내용물을 기록하고 저장하기
  1. 파일을 읽어와 헤더와 콘텐츠를 분리하기
  1. 첫 줄에만 헤더를 삽입하고 그 다음 줄부터는 콘텐츠만 삽입하기

#### 2. 헤더를 분리하는 방법 (텍스트 스플릿)
- ```split```

In [3]:
temp = "abcd : efgh"

In [4]:
temp.split(":")

['abcd ', ' efgh']

In [5]:
# 공백 제거
temp.split(" : ")

['abcd', 'efgh']

In [9]:
# 공백 제거
[elem.strip() for elem in temp.split(":")]

['abcd', 'efgh']

#### 3. 헤더를 맨 첫줄에만 삽입하고 그 다음부터는 삽입하지 않는 방법
1. for 문이 시작하기 전에 미리 헤더를 삽입, for 문 안에서는 데이터만 삽입하는 방법
1. **헤더가 삽입했는지 아닌지를 따로 기록하는 방법
  - 아직 헤더가 삽입되지 않았다고 적혀 있다면 헤더를 삽입
  - 헤더가 이미 삽입되었다고 적혀 있다면 건너뛰기

**: 본 예제에서 사용되는 방법

In [12]:
# full code

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.23
"""

# import
import time
import os

#
print("Process Start.")
start_time = time.time()

#
directory = "personal_info"
outfile_name = "merged_ID.csv"

#
out_file = open(outfile_name, "w")
input_files = os.listdir(directory)

#
headers = []
outfile_has_header = False

#
for filename in input_files:

    # exception
    if ".txt" not in filename:
        continue
    
    #
    file = open(directory + "/" + filename)

    #
    contents = []

    for line in file:

        # split headers and contents
        if ":" in line:
            splits = line.split(":")
            contents.append(splits[-1].strip())

            ## (포인트) contents 개수가 많다면, 미처 발견 못한 headers 요소가 있다는 것
            if len(contents) > len(headers):
                headers.append(splits[0].strip())
    
    #
    if not outfile_has_header:
        header = ", ".join(headers)
        out_file.write(header)
        outfile_has_header = True
    
    #
    new_line = ", ".join(contents)
    out_file.write("\n" + new_line)

    #
    file.close()

#
out_file.close()

#
print("Process Done.")
end_time = time.time()
print("Peter's code: " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code: 0.031876564025878906 seconds.
